In [1]:
import random
import time
import os
import networkx as nx
import numpy as np
from copy import deepcopy
from tabulate import tabulate
from matplotlib import pyplot as plt

# Omogućava prikaz grafikona unutar Notebook-a
%matplotlib inline

In [2]:
def maxNumOfColours(graph):
    maxDegree = 0
    for i in graph:
        degree = len(list(graph.neighbors(i)))
        if degree > maxDegree:
            maxDegree = degree
    return maxDegree

def readGraph(filePath):
    graph = nx.Graph()
    with open(filePath) as f:
        lines = f.readlines()
        numNodes = int(lines[0].split()[2])
        # Normalizacija grana: sorted osigurava da je (1,2) isto što i (2,1)
        rawEdges = [tuple(map(int, line.split()[1:])) for line in lines[1:] if line.startswith('e')]
        edges = [tuple(sorted(edge)) for edge in rawEdges]
        graph.add_nodes_from(range(1, numNodes + 1))
        graph.add_edges_from(edges)
    return graph

def getFitness(chromosome, edges, maxNode):
    conflicts = 0
    nodeMap = {i: [] for i in range(1, maxNode + 1)}
    for i, color in enumerate(chromosome):
        u, v = edges[i]
        nodeMap[u].append(color)
        nodeMap[v].append(color)
    for node in nodeMap:
        colors = nodeMap[node]
        if len(colors) > 1:
            conflicts += (len(colors) - len(set(colors)))
    return conflicts

In [3]:
def selectionTournament(population, fitnesses, k=3):
    participants = random.sample(list(zip(population, fitnesses)), k)
    return min(participants, key=lambda x: x[1])[0]

def selectionRoulette(population, fitnesses):
    # Što manji fitnes, to veća šansa (proporcionalno 1/f)
    invertedFitness = [1.0 / (f + 1e-6) for f in fitnesses]
    total = sum(invertedFitness)
    pick = random.uniform(0, total)
    current = 0
    for i, f in enumerate(invertedFitness):
        current += f
        if current > pick:
            return population[i]
    return population[0]

In [4]:
def crossoverUniform(p1, p2):
    return [p1[i] if random.random() < 0.5 else p2[i] for i in range(len(p1))]

def crossoverSinglePoint(p1, p2):
    point = random.randint(1, len(p1) - 1)
    return p1[:point] + p2[point:]

In [5]:
def mutationSmart(chromosome, edges, maxNode, delta):
    idx = random.randrange(len(chromosome))
    u, v = edges[idx]
    forbidden = set()
    for i, (n1, n2) in enumerate(edges):
        if i != idx and (n1 in (u, v) or n2 in (u, v)):
            forbidden.add(chromosome[i])
    available = list(set(range(1, delta + 2)) - forbidden)
    if available:
        chromosome[idx] = random.choice(available)
    else:
        chromosome[idx] = random.randint(1, delta + 1)
    return chromosome

def mutationRandom(chromosome, edges, maxNode, delta):
    idx = random.randrange(len(chromosome))
    chromosome[idx] = random.randint(1, delta + 1)
    return chromosome

In [6]:
def runGaModular(graph, maxIters, popSize, selFunc, crossFunc, mutFunc):
    edges = list(graph.edges())
    if not edges: return [], 0
    maxNode = max(graph.nodes())
    delta = maxNumOfColours(graph)
    numEdges = len(edges)
    population = [[random.randint(1, delta + 1) for _ in range(numEdges)] for _ in range(popSize)]
    
    for gen in range(maxIters):
        fitnesses = [getFitness(c, edges, maxNode) for c in population]
        if min(fitnesses) == 0:
            break
        popFit = sorted(zip(population, fitnesses), key=lambda x: x[1])
        newPop = [popFit[0][0], popFit[1][0]]  # Elitizam: čuvamo 2 najbolja
        
        while len(newPop) < popSize:
            p1 = selFunc(population, fitnesses)
            p2 = selFunc(population, fitnesses)
            child = crossFunc(p1, p2)
            if random.random() < 0.3:
                child = mutFunc(child, edges, maxNode, delta)
            newPop.append(child)
        population = newPop
        
    finalFitnesses = [getFitness(c, edges, maxNode) for c in population]
    bestIdx = np.argmin(finalFitnesses)
    return population[bestIdx], finalFitnesses[bestIdx]

def drawGraphResult(graph, solution, filePath, bestCombo, bestTime):
    plt.figure(figsize=(10, 7))
    pos = nx.spring_layout(graph, seed=42)
    nx.draw(graph, pos, edge_color=solution, width=3, with_labels=True,
            node_color='lightblue', node_size=500, edge_cmap=plt.cm.rainbow)
    plt.title(f"Graf: {filePath}\nPobednik (Najbrži): {bestCombo} ({bestTime:.4f}s)", 
              fontsize=12, fontweight='bold')
    plt.show()

In [7]:
def processAndVisualize(filePath):
    graph = readGraph(filePath)
    selections = [selectionTournament, selectionRoulette]
    crossovers = [crossoverUniform, crossoverSinglePoint]
    mutations = [mutationSmart, mutationRandom]
    results = []
    bestTime = float('inf')
    bestSol = None
    bestComboName = ""
    
    print(f"\n>>> ANALIZA GRAFA: {filePath}")

    for sel in selections:
        for cross in crossovers:
            for mut in mutations:
                sName = "Tour" if "Tournament" in sel.__name__ else "Roul"
                cName = "Unif" if "Uniform" in cross.__name__ else "Sing"
                mName = "Smart" if "Smart" in mut.__name__ else "Rand"
                comboName = f"{sName}+{cName}+{mName}"
                
                startTime = time.time()
                sol, fit = runGaModular(graph, 300, 50, sel, cross, mut)
                dt = time.time() - startTime
                
                results.append([comboName, fit, f"{dt:.4f}s"])
                
                if fit == 0 and dt < bestTime:
                    bestTime, bestSol, bestComboName = dt, sol, comboName
                elif bestSol is None:
                    bestTime, bestSol, bestComboName = dt, sol, comboName

    print(tabulate(results, headers=["Kombinacija", "Konflikti", "Vreme"], tablefmt="fancy_grid"))
    drawGraphResult(graph, bestSol, filePath, bestComboName, bestTime)

# Pronalazi sve .txt fajlove u trenutnom folderu i pokreće analizu
files = [f for f in os.listdir(".") if f.endswith('.txt')]
for file in sorted(files):
    processAndVisualize(file)